### Train the GNN on the lastfm dataset

In [1]:
from recbole_gnn.quick_start import run_recbole_gnn

In [28]:
config_path = 'lastfm_config.yaml'
results = run_recbole_gnn(dataset='lastfm', model='MHCN', config_file_list=[config_path])

25 Nov 16:31    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/lastfm
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 100
train_batch_size = 4096
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 50
clip_grad_norm =

 None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = NDCG@10
valid_metric_bigger = True
eval_batch_size = 409600000
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user_id
ITEM_ID_FIELD = artist_id
RATING_FIELD = rating
TIME_FIELD = timestamp
seq_len = None
LABEL_FIELD = label
threshold = None
NEG_PREFIX = neg_
load_col = {'inter': ['user_id', 'artist_id'], 'net': ['source_id', 'target_id']}
unload_col = None
unused_col = None
additional_feat_suffix = None
rm_dup_inter = None
val_interval = None
filter_inter_by_user_or_item = True
user_inter_num_interval = [0,inf)
item_inter_num_interval = [0,inf)
alias_of_user_id = None
alias_of_item_id = None
alias_of_entity_id = None
alias_of_relation_id = None
preload_we

### Load Model after training

In [32]:
# from recbole.quick_start.quick_start import load_data_and_model
# This function does not work for recbole_gnn models

In [3]:
from logging import getLogger
from recbole.utils import init_logger, init_seed, set_color
from recbole_gnn.utils import create_dataset, data_preparation, get_model

# This function is from the recbole.quick_start.quick_start module, but adapted to the recbole_gnn lib
def load_data_and_model(model_file):
    r"""Load filtered dataset, split dataloaders and saved model.

    Args:
        model_file (str): The path of saved model file.

    Returns:
        tuple:
            - config (Config): An instance object of Config, which record parameter information in :attr:`model_file`.
            - model (AbstractRecommender): The model load from :attr:`model_file`.
            - dataset (Dataset): The filtered dataset.
            - train_data (AbstractDataLoader): The dataloader for training.
            - valid_data (AbstractDataLoader): The dataloader for validation.
            - test_data (AbstractDataLoader): The dataloader for testing.
    """
    import torch

    checkpoint = torch.load(model_file)
    config = checkpoint["config"]
    init_seed(config["seed"], config["reproducibility"])
    init_logger(config)
    logger = getLogger()
    logger.info(config)

    dataset = create_dataset(config)
    logger.info(dataset)
    train_data, valid_data, test_data = data_preparation(config, dataset)

    init_seed(config["seed"], config["reproducibility"])
    model = get_model(config["model"])(config, train_data._dataset).to(config["device"])
    model.load_state_dict(checkpoint["state_dict"])
    model.load_other_parameter(checkpoint.get("other_parameter"))

    return config, model, dataset, train_data, valid_data, test_data

In [4]:
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
    model_file='saved/MHCN-Nov-25-2023_16-31-04.pth',
)

03 Dec 11:39    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/lastfm
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 100
train_batch_size = 4096
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 50
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = NDCG@10
valid_metric_bigger = True
eval_batch_size = 409600000
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq

### Make predictions

In [12]:
from recbole.data.interaction import Interaction
from recbole.evaluator import eval_tools

ImportError: cannot import name 'eval_tools' from 'recbole.evaluator' (/Users/hugobouy/miniconda3/envs/ml_project/lib/python3.11/site-packages/recbole/evaluator/__init__.py)

In [ ]:
# Make predictions/recommendations for the test interactions
topk = 10  # Number of items to recommend
user_ids = test_data[Interaction.USER_ID]
item_ids = test_data[Interaction.ITEM_ID]
with torch.no_grad():
    scores = model.predict(user_ids, item_ids)

# Get top-k recommended items for each user
topk_items = eval_tools.get_topk_items(scores, topk)

# Print or use the recommendations as needed
print("Top-10 recommendations for each user:")
for user, items in topk_items.items():
    print(f"User {user}: {items}")

### Tests

In [83]:
user_features = dataset.get_user_feature()
print(user_features['user_id'])

tensor([   0,    1,    2,  ..., 1890, 1891, 1892])


In [77]:
user_id = 2  # replace with the specific user ID
user_interaction = dataset.get_user_feature()[user_id]  # get the data for the specific user
user_interaction = user_interaction.to(config['device'])  # move the interaction to the device
prediction = model.predict(user_interaction)  # make predictions for the user
print(prediction)


KeyError: 'artist_id'

In [58]:
model.eval()
for batch_idx, interaction in enumerate(train_data):
    print(interaction[0])
    interaction = interaction.to(config['device'])
    prediction = model.predict(interaction)
    print(prediction[0])
    break

The batch_size of interaction: 1
    user_id, torch.Size([]), cpu, torch.int64
    artist_id, torch.Size([]), cpu, torch.int64
    neg_artist_id, torch.Size([]), cpu, torch.int64




tensor(5.4047, grad_fn=<SelectBackward0>)


In [43]:
from dig.xgraph.method import SubgraphX

explainer = SubgraphX(model, num_classes=1, device=config['device'], explain_graph=False, reward_method='nc_mc_l_shapley')